# Les bases de pandas pour faire interagir Excel avec Python

L'objectif de ce module de 3h est d'enseigner les bases de la librairie Pandas et de montrer aux étudiants qu'il est possible de faire en quelques lignes de code des opérations qui demandent beaucoup plus de temps avec Excel. 

Ce module doit contenir une partie d'introduction au module pandas, une partie où il leur est montré comment combiner quasi automatiquement des jeux de données avec pandas, et une dernière partie où il leur est montré comment faire parler très rapidement des données grâce à des opérations de filtrage. 

Propose moi, un déroulé de cours avec des bouts de code à chaque instant.

pandas makes tasks that are particularly painful in Excel easier, faster, and less error-prone.

the main components of a DataFrame: index, columns, and data.

Let’s now re-create this DataFrame from scratch without reading it from the Excel file: one way of creating a DataFrame is to provide the data as a nes‐ ted list, along with values for columns and index:

By doing df.reset_index().set_index("name"), you are using method chaining:

Real-world data hardly gets served on a silver platter, so before working with it, you need to clean it and bring it into a digestible form. 

Imagine that you need to anonymize all names of people who are below 20 years old or from the USA:

In [41]: df2.replace("USA", "U.S.")

However, the true power of pandas is its automatic data alignment mechanism: when you use arithmetic operators with more than one DataFrame, pandas automatically aligns them by their columns and row indices.

DataFrames offer the applymap method, which will apply a function to every individ‐ ual element, something that is useful if there are no NumPy ufuncs available.

In [69]: rainfall.applymap(lambda x: f"{x:,.2f}")

Combining different datasets in Excel can be a cumbersome task and typically involves a lot of VLOOKUP formulas


The special and very useful feature of concat is that it accepts more than two Data‐ Frames. We will use this in the next chapter to make a single DataFrame out of multi‐ ple CSV files:
pd.concat([df1, df2, df3, ...])

In [88]: df.groupby(["continent"]).mean()

A popular way to get statistics per group in Excel is to use pivot tables.

 In [92]: pivot = pd.pivot_table(sales,
                                    index="Fruit", columns="Region",
                                    values="Revenue", aggfunc="sum",
                                    margins=True, margins_name="Total")
pivot
    Out[92]: Region   North  South  Total
             Fruit
             Apples     NaN  10.55  10.55
             Bananas   31.3   5.90  37.20
             Oranges   25.4  22.00  47.40
             Total     56.7  38.45  95.15

Working with aggregated statistics helps you understand your data, but nobody likes to read a page full of numbers.

## Selectionner les données

In [4]:
import pandas as pd

In [10]:
df_reference = pd.read_excel("referentiel.xlsx", header = None)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,NaN,Pays_Cod,Pays,Région,Sous-Région,Code Sous-Région,NaN,Product_Ref,Produit,Catégorie de produit,Couleur,Créé le,Prix d'achat,Prix vente HT
1,NaN,BGR,Bulgarie,Europe,Europe de l'Est,EUE,NaN,P00147,Robe,Haut-Et-Bas,marron,2018-01-01 00:00:00,7.2,9
2,NaN,CZE,République Tchèque,Europe,Europe de l'Est,EUE,NaN,P00249,Chemise,Haut,blanc,2018-06-01 00:00:00,11.34,14
3,NaN,HUN,Hongrie,Europe,Europe de l'Est,EUE,NaN,P00565,Robe,Haut-Et-Bas,orange,2017-05-01 00:00:00,13.05,15
4,NaN,MDA,République de Moldavie,Europe,Europe de l'Est,EUE,NaN,P00575,Pyjama,Haut-Et-Bas,blanc,2018-12-01 00:00:00,5.32,7


## Agréger toutes les données

In [ ]:
...

In [ ]:
...

## Interroger les données (dans tous les sens)

...

...

## Quelque chose de cool pour synthétiser la connaissance des données acquises

...

...

# Mise en pratique

Quand on ouvre des fichiers Excel pour la première fois, il est souvent difficile de leur donner du sens. Les données que l'on reçoit sont rarement calibrées pour notre manière personnelle de comprendre les choses. 

Voyez pandas comme un outil ultra puissant qui va permettre de customiser des données pour vous. 

Mettons que votre boss vient de vous envoyer ces deux fichier Excel 👇

**Fichier 1**

<img src="media/ventes.png" width="500"/>

**Fichier 2**

<img src="media/referentiel.png" width="500"/>

Vous voulez comprendre **tout ce qu'il est possible de comprendre** sur l'activité de l'entreprise à partir de ces deux fichiers (en un temps record).

In [3]:
import pandas as pd